In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, dataloader
import torch.nn.functional as F
import json
import pickle
import re
from typing import List, Union
from tqdm import tqdm
import math
import os
import numpy as np


In [14]:
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device('mps')
    else:
        return torch.device('cpu')

device = get_device()
device

device(type='mps')

In [15]:
PAD = "<PAD>"
UNK = "<UNK>"

with open("wiki2vec/dump_t.pkl", "rb") as f:
    word2idx, idx2word, embeddings = pickle.load(f)
    
WHITESPACE = "<WS>"
word2idx[WHITESPACE] = len(word2idx)
idx2word[word2idx[WHITESPACE]] = WHITESPACE
dim = embeddings.shape[1]
ws_emb = torch.randn(1, dim)
embeddings = torch.cat([embeddings, ws_emb], dim=0)
embeddings.shape

torch.Size([4530033, 300])

In [16]:
class TaskC(Dataset):
    def __init__(self, split="train", enrich=False, max_len=2000, word2idx=None):
        super().__init__()
        self.split = split
        self.enrich = enrich
        self.max_len = max_len
        self.word2idx = word2idx
        self.data = self.load_data()
    
    def load_data(self):
        f = "data/subtaskC_train.jsonl" if self.split == "train" else "data/subtaskC_dev.jsonl"
        data = []
        with open(f, "r", encoding="utf-8") as f:
            for line in f:
                parsed = json.loads(line)
                data.append(parsed)
                
        if self.split == "train" and self.enrich:
            with open("data/subtaskA_train_monolingual.jsonl", "r", encoding="utf-8") as f:
                for line in f:
                    parsed = json.loads(line)
                    text = parsed["text"]
                    is_machine = parsed["label"] == 1
                    label = 0 if is_machine else -1
                    data.append({
                        "text": text,
                        "label": label
                    })
        return data
    
    def _clean(self, word):
        # all non alphanumeric
        replace = re.compile(r"[^a-zA-Z0-9\-]")
        word = replace.sub("", word)
        return word.lower().strip()
    
    def _is_whitespace(self, word):
        pat = re.compile(r"^\s*$")
        return pat.match(word) is not None

    def tokenize(self, text) -> List[int]:
        tokens = text.split(" ")
        ids = []
        for token in tokens:
            token = self._clean(token)
            if self._is_whitespace(token):
                ids.append(self.word2idx[WHITESPACE])
            elif token in self.word2idx:
                ids.append(self.word2idx[token])
            else:
                ids.append(self.word2idx[UNK])
        if len(ids) > self.max_len:
            ids = ids[:self.max_len]
        elif len(ids) < self.max_len:
            ids = ids + ([self.word2idx[PAD]] * (self.max_len - len(ids)))
        return torch.tensor(ids, dtype=torch.long)
    
    def get_label_vector(self, boundary:int):
        base = [0] * self.max_len
        if boundary > -1:
            for i in range(boundary, self.max_len):
                base[i] = 1
        return torch.tensor(base, dtype=torch.long)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        data = self.data[index]
        text = self.tokenize(data["text"])
        label = self.get_label_vector(data["label"])
        return text, label

In [17]:
class BiLSTM_Labeller(nn.Module):
    def __init__(
        self, 
        pretrained_embeddings = None,
        hidden_size = 256,
        num_layers = 1
    ) -> None:
        
        super().__init__()
        self.hidden_size = hidden_size
        
        self.emb = nn.Embedding.from_pretrained(pretrained_embeddings)
        self.emb.cpu()
        self.input_size = self.emb.weight.shape[1]
        
        self.lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
        )
        self.lstm.to(device)
        # multiply by 2 because of bidirectional
        self.classifier_head = nn.Linear(self.hidden_size * 2, 2)
        self.classifier_head.to(device)
        
    def forward(self, input_ids):
        inputs: torch.Tensor = self.emb(input_ids)
        inputs = inputs.to(device)
        outputs, _ = self.lstm(inputs)
        logits = self.classifier_head(outputs)
        predicted = F.log_softmax(logits, dim=-1)
        return predicted
    
    def predict(self, input_ids) -> List[int]:
        predicted = self.forward(input_ids)
        predicted = torch.argmax(predicted, dim=-1)
        p = predicted.cpu().numpy().tolist()
        r = []
        for item in p:
            try:
                r.append(item.index(1))
            except ValueError:
                r.append(len(item) // 2)
        return r
    
model = BiLSTM_Labeller(
    pretrained_embeddings=embeddings
)



In [18]:
def get_start_position(pred: torch.Tensor) -> int:
    pred_np: np.ndarray = pred.detach().numpy()
    result = []
    for labels in pred_np:
        try:
            result.append(labels.tolist().index(1))
        except ValueError:
            result.append(len(labels) // 2)
    return result

In [19]:
taskC_dev = TaskC(split="dev", word2idx=word2idx)
dev_dataloader = dataloader.DataLoader(taskC_dev, batch_size=32, shuffle=False)

def eval(model):
    pred = []
    gold = []
    with torch.no_grad():
        for input_ids, labels in dev_dataloader:
            labels = get_start_position(labels)
            predicted = model.predict(input_ids)
            pred.extend(predicted)
            gold.extend(labels)
    distances = [abs(pred[i] - gold[i]) for i in range(len(pred))]
    print("MEAN ABSOLUTE DISTANCE:", sum(distances) / len(distances))
    return pred, gold



In [20]:
def make_checkpoint(model, optimizer, epoch):
    try:
        os.mkdir("checkpoints")
    except FileExistsError:
        pass
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch": epoch,
    }
    torch.save(checkpoint, f"checkpoints/checkpoint_{epoch}.pt")

In [21]:
def train(model, dataloader, optimizer, criterion, epochs=10):
    model.train()
    
    for epoch in range(1, epochs+1): 
        losses = []
        with tqdm(dataloader, total=len(dataloader)) as pbar:
            for batch in dataloader:
                optimizer.zero_grad()
                input_ids, labels = batch
                # do not put inputs to cuda, because emb is on cpu
                labels = labels.to(device)
                
                outputs = model(input_ids)
                # put outputs in shape (batch_size * seq_len, 2)
                outputs = outputs.reshape(-1, 2)
                # put labels in shape (batch_size * seq_len)
                labels = labels.reshape(-1)
                loss = criterion(outputs, labels)
                losses.append(loss.item())
                loss.backward()
                optimizer.step()
                pbar.update(1)
        print(f"\n---- EVALUATING MODEL at epoch {epoch} ----")
        print("LOSS:", sum(losses) / len(losses))
        eval(model)
        print("------------------------------------------\n")
        # make_checkpoint(model, optimizer, epoch)

In [22]:
loader = dataloader.DataLoader(
    TaskC(word2idx=word2idx, split="train"),
    batch_size=32,
    shuffle=True,
)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

train(model, loader, optimizer, criterion)

  0%|          | 0/115 [00:00<?, ?it/s]

100%|██████████| 115/115 [02:02<00:00,  1.07s/it]


---- EVALUATING MODEL at epoch 1 ----
LOSS: 0.09684148352269245
MEAN ABSOLUTE DISTANCE: 64.409900990099


100%|██████████| 115/115 [02:01<00:00,  1.06s/it]


---- EVALUATING MODEL at epoch 2 ----
LOSS: 0.0394621753498264
MEAN ABSOLUTE DISTANCE: 22.102970297029703


...

---- EVALUATING MODEL at epoch 8 ----
LOSS: 0.015913577426386917
MEAN ABSOLUTE DISTANCE: 15.146534653465347


100%|██████████| 115/115 [02:01<00:00,  1.06s/it]


---- EVALUATING MODEL at epoch 9 ----
LOSS: 0.014954867282801348
MEAN ABSOLUTE DISTANCE: 13.24950495049505


100%|██████████| 115/115 [02:01<00:00,  1.06s/it]


---- EVALUATING MODEL at epoch 10 ----
LOSS: 0.014239070021911807
MEAN ABSOLUTE DISTANCE: 14.752475247524753
